Задание:
> Run manually configured Kafka Cluster with 3+ instances
> 
> Use either Zookeper or KRaft
> 
> Create simulated stream workload (logs, payments, clicks, etc.)
> 
> Create consumers
> 
> Simulate outage of minority, observe availability
> 
> Simulate outage of majority, observe unavailability

Хочу Raft и отдельную ноду Кафки в качестве контроллера. Интересно посмотреть, что произойдёт с кластером при падении контроллера.

Протокол CONTROLLER используется только для связи между контроллерами (выборы лидера, метаданные). Для обычного брокера (`process.roles=broker`) не нужен CONTROLLER-листенер, но нужно знать, где контроллера можно найти (`controller.quorum.voters`). Брокер подключается к контроллеру через его CONTROLLER-листенер (порт 19093 по умолчанию) для:
* Регистрации при старте
* Регулярных heartbeat-ов
* Получения команд (назначение лидера партиции, обновление метаданных: топики, лидеры партиций)
* Уведомляет о статусе in-sync реплик

Обратный канал отсутствует. Контроллер не подключается к брокеру — все команды отправляются как ответы на heartbeat-ы.

Конфиги контроллеру и брокеру нужны разные. Брокерам оставляю то, что связано с топиками, убираю у них controller-слушателя. Насчёт некоторых настроек было сложно определить, кому они нужны - чистому контроллеру или чистому брокеру:

* `unclean.leader.election.enable` нужна брокерам несмотря на то, что выбором лидера занимается контроллер. Контроллер всегда старается назначить лидера из ISR (in-sync replicas), независимо от `unclean.leader.election.enable`. Если реплики из ISR живы — контроллер выбирает одну из них. Если ISR пуст - контроллер смотрит значение `unclean.leader.election.enable` на брокерах. Так что этот параметр нужен только на брокерах и не нужен на контроллере
* `auto.leader.rebalance.enable` нужна брокерам. Брокеры следят за своей собственной нагрузкой и запрашивают ребалансировку у контроллеров, если слишком во многих партициях они назначены лидерами. Контроллеры только исполняют ребалансировку, но не инициируют (UPD позднее окажется, что это не так)
* `group.initial.rebalance.delay.ms` нужна брокерам. Ребалансировки в консьюмер-группах инициируют брокеры - брокеры, назначенные координаторами консьюмер-групп. У каждой консьюмер-группы есть свой координатор, выбранный из брокеров (а не контроллеров)
* `default.replication.factor` нужен на контроллере, иначе просто не работает репликация

In [5]:
!./kafka/bin/kafka-storage.sh format -t $(./kafka/bin/kafka-storage.sh random-uuid) -c ./kafka/a-config/controller.1.properties

Formatting metadata directory ./tmp/c4 with metadata.version 3.9-IV0.


Затем запускаю трёх брокеров и одного контроллера и создаю топик `page-views`

In [74]:
!./kafka/bin/kafka-topics.sh --create --topic user-events --partitions 8 --bootstrap-server localhost:19092,localhost:19091,localhost:19090

Created topic user-events.


In [75]:
!./kafka/bin/kafka-topics.sh --describe --topic user-events --bootstrap-server localhost:19092,localhost:19091,localhost:19090

Topic: user-events	TopicId: 3lls2lKPSl2Ljop6-DkC9Q	PartitionCount: 8	ReplicationFactor: 3	Configs: min.insync.replicas=2,unclean.leader.election.enable=false
	Topic: user-events	Partition: 0	Leader: 1	Replicas: 1,2,3	Isr: 1,2,3	Elr: 	LastKnownElr: 
	Topic: user-events	Partition: 1	Leader: 2	Replicas: 2,3,1	Isr: 2,3,1	Elr: 	LastKnownElr: 
	Topic: user-events	Partition: 2	Leader: 3	Replicas: 3,1,2	Isr: 3,1,2	Elr: 	LastKnownElr: 
	Topic: user-events	Partition: 3	Leader: 3	Replicas: 3,2,1	Isr: 3,2,1	Elr: 	LastKnownElr: 
	Topic: user-events	Partition: 4	Leader: 2	Replicas: 2,1,3	Isr: 2,1,3	Elr: 	LastKnownElr: 
	Topic: user-events	Partition: 5	Leader: 1	Replicas: 1,3,2	Isr: 1,3,2	Elr: 	LastKnownElr: 
	Topic: user-events	Partition: 6	Leader: 1	Replicas: 1,2,3	Isr: 1,2,3	Elr: 	LastKnownElr: 
	Topic: user-events	Partition: 7	Leader: 2	Replicas: 2,3,1	Isr: 2,3,1	Elr: 	LastKnownElr: 


Пробую писать в топик c `acks=all` в трёх условиях:
* когда весь кластер жив
* когда не отвечает один инстанс брокера
* когда нет достаточного количества in-sync реплик (в моей конфигурации это происходит когда нет двух из трёх инстансов)

![image.png](pic/_1.png)

1. Когда весь кластер жив, всё хорошо. Параметр `acks=all` не означает "ждать подтверждения от всех созданных реплик" (`default.replication.factor`), а именно от всех реплик, которые сейчас в ISR. Первоначально в топике три ISR, так что если все продьюсеры будут записывать с `acks=all`, то им всегда придётся дожидаться записи на три реплики. Но если появится один продьюсер с `acks=1` или `acks=0`, он нарушит ISR=3. Лидер партиции будет записывать сообщения быстрее, чем другие реплики - реплицировать. В этом случае в моменте ISR может быть равно 1 или 2, и продьюсеры с `acks=all` часто будут ждать записи не на три реплики, а на две, как заложено в `min.insync.replicas`.
2. Когда падает один брокер, то продьюсеру не удаётся запись в те партиции, где упавший брокер был лидером. Он перезапрашивает метаданные у любого живого брокера, узнает нового лидера партиции и направляет сообщение ему. Так что продьюсер замечает исчезновение одного из брокеров, но с этим справляется.
3. Когда падает два из трёх брокеров, запись останавливается, потому что становится невозможно записать сообщение, не нарушив `min.insync.replicas=2`.

Если заменить `acks=all` на `acs=1`, достаточно будет подтверждения записи на лидер-реплику - единственного выжившего брокера. 

![image.png](pic/_2.png)

Когда упавшие реплики вернутся в строй, они скопируют себе с лидера те сообщения, которые появились в топике, пока что они были оффлайн. Пока репликация не будет завершена, они не смогут взять лидерство над партицией из-за настройки `unclean.leader.election.enable=false`

Поставлю `auto.leader.rebalance.enable=true` и `leader.imbalance.check.interval.seconds=1000` на контроллере и `auto.leader.rebalance.enable=false` на брокерах, чтобы увидеть, произойдёт ребаланс или нет после возвращения брокеров в строй.

In [93]:
!./kafka/bin/kafka-topics.sh --describe --topic user-events --bootstrap-server localhost:19092,localhost:19091,localhost:19090

Topic: user-events	TopicId: 3lls2lKPSl2Ljop6-DkC9Q	PartitionCount: 8	ReplicationFactor: 3	Configs: min.insync.replicas=2,unclean.leader.election.enable=false
	Topic: user-events	Partition: 0	Leader: 1	Replicas: 1,2,3	Isr: 1	Elr: 	LastKnownElr: 
	Topic: user-events	Partition: 1	Leader: 1	Replicas: 2,3,1	Isr: 1	Elr: 	LastKnownElr: 
	Topic: user-events	Partition: 2	Leader: 1	Replicas: 3,1,2	Isr: 1	Elr: 	LastKnownElr: 
	Topic: user-events	Partition: 3	Leader: 1	Replicas: 3,2,1	Isr: 1	Elr: 	LastKnownElr: 
	Topic: user-events	Partition: 4	Leader: 1	Replicas: 2,1,3	Isr: 1	Elr: 	LastKnownElr: 
	Topic: user-events	Partition: 5	Leader: 1	Replicas: 1,3,2	Isr: 1	Elr: 	LastKnownElr: 
	Topic: user-events	Partition: 6	Leader: 1	Replicas: 1,2,3	Isr: 1	Elr: 	LastKnownElr: 
	Topic: user-events	Partition: 7	Leader: 1	Replicas: 2,3,1	Isr: 1	Elr: 	LastKnownElr: 


Жду 10 секунд, выполняю ещё раз:

In [98]:
!./kafka/bin/kafka-topics.sh --describe --topic user-events --bootstrap-server localhost:19092,localhost:19091,localhost:19090

Topic: user-events	TopicId: 3lls2lKPSl2Ljop6-DkC9Q	PartitionCount: 8	ReplicationFactor: 3	Configs: min.insync.replicas=2,unclean.leader.election.enable=false
	Topic: user-events	Partition: 0	Leader: 1	Replicas: 1,2,3	Isr: 1,2,3	Elr: 	LastKnownElr: 
	Topic: user-events	Partition: 1	Leader: 2	Replicas: 2,3,1	Isr: 1,2,3	Elr: 	LastKnownElr: 
	Topic: user-events	Partition: 2	Leader: 3	Replicas: 3,1,2	Isr: 1,2,3	Elr: 	LastKnownElr: 
	Topic: user-events	Partition: 3	Leader: 3	Replicas: 3,2,1	Isr: 1,2,3	Elr: 	LastKnownElr: 
	Topic: user-events	Partition: 4	Leader: 2	Replicas: 2,1,3	Isr: 1,2,3	Elr: 	LastKnownElr: 
	Topic: user-events	Partition: 5	Leader: 1	Replicas: 1,3,2	Isr: 1,2,3	Elr: 	LastKnownElr: 
	Topic: user-events	Partition: 6	Leader: 1	Replicas: 1,2,3	Isr: 1,2,3	Elr: 	LastKnownElr: 
	Topic: user-events	Partition: 7	Leader: 2	Replicas: 2,3,1	Isr: 1,2,3	Elr: 	LastKnownElr: 


Значит, настройка ребалансировки лидера нужна на контроллерах и неверно было утрерждение, что именно брокеры инициируют ребалансировку. Дизлайк Deepseek-у! Вероятно, и утверждение про то, что `unclean.leader.election.enable` нужна только на брокере, тоже не совсем верно. Но я не нашёл способа как это проверить - надо было бы остановить репликацию на возвращённом в строй брокере и убедится, что он не становится лидером, а я не уверен, что можно командой остановить репликацию. Можно как-то замедлить сеть или заблочить порт, но я не стал заморачиваться. 

Теперь добавлю двух консьюмеров-одногруппников и выключу контроллера.

![image.png](pic/_3.png)

![image.png](pic/_4.png)

Запись в топик продолжилась, чтение из топиков продолжилось, кластер целиком доступен, но в логах брокеров началось сумасшествие. Команда `./kafka/bin/kafka-topics.sh --describe --topic user-events --bootstrap-server localhost:19092,localhost:19091,localhost:19090` теперь зависает и не выполняется несмотря на то, что в `--bootstrap-server` перечислены живые хосты. Предполагаю, что другие команды (например, команда сбалансировать лидеров партиций) тоже подвиснут. Репликация не остановилась: я делаю этот вывод, исходя из того, что продьюсеру с `acks=all` удаётся запись в топик, несмотря на отсутствие контроллера. Пробую убить одного брокера через `SIGTERM`, но брокер не хочет закрываться. Узнаю его pid и убиваю силой через `kill -9`.

После этого продьюсер и консьюмер останавливаются, хотя ISR=2, и по идее можно было бы писать дальше. Спустя минуту у продьюсера появляются таймауты от живых нод Кафки: `localhost:19091/2: Timed out ProduceRequest in flight`, что говорит о том, что живые брокеры перестали принимать запросы на запись. Судя по логам, они бесконечно пытаются подключиться к упавшему брокеру из треда `ReplicaFetcher` и к котроллеру из `NodeToControllerChannelManager`. Притом на остановку процессов не влияет то, была ли выключенная нода лидером в каких-то партициях топика. Я повторял эксперимент по разному - и от того, была ли выключенная нода лидером, зависит лишь то, появятся ли сразу после её выключения ошибки `Disconnected` в логах продьюсера и консьюмера (на скриншоте ниже они есть).

Если запустить нового консьюмера из новой группы, он попытается установить соединение с выключенным брокером даже если убрать его из bootstrap-servers. Видимо, за отсутствием контроллера метаданные про топик консьюмер получил от одного из живых брокеров, а у него эти метаданные (тоже из-за отсутствия контроллера) неактуальные - и ведут на выключенную ноду.

Предполагаю, что без контроллера кластер Кафки может штатно работать только до тех пор, пока не случится какое-то событие, которое потребует участия контроллера. Если такое событие происходит, гаснут и продьюсеры, и консьюмеры. 

![image.png](pic/_5.png)

Если в этот момент вернуть контроллера, то оставшиеся в живых два брокера, которых я тоже пытался убить, наконец позволяют себе упасть. В логах от них остаются такие сообщения:
```
[2025-07-20 13:15:52,896] INFO [BrokerLifecycleManager id=2] The broker is in PENDING_CONTROLLED_SHUTDOWN state, still waiting for the active controller. (kafka.server.BrokerLifecycleManager)
[2025-07-20 13:15:53,294] ERROR [BrokerServer id=2] Timed out waiting for the controller to approve controlled shutdown (kafka.server.BrokerServer)
[2025-07-20 13:15:53,294] INFO [BrokerLifecycleManager id=2] beginShutdown: shutting down event queue. (org.apache.kafka.queue.KafkaEventQueue)
[2025-07-20 13:15:53,294] INFO [BrokerLifecycleManager id=2] Transitioning from PENDING_CONTROLLED_SHUTDOWN to SHUTTING_DOWN. (kafka.server.BrokerLifecycleManager)
```

Они ждали контроллера, дождались, но контроллер не успел им ответить (`Timed out waiting for the controller to approve`). В конечном итоге, контроллеру тоже нужно время на запуск, прежде чем он будет готов обрабатывать запросы с кластера.

Если повторить инцидент, но не гасить брокеров, то после возвращения контроллера в кластер (в котором все три брокера живы) всё станет снова хорошо. Возобновятся и запись, и чтение.